In [3]:
import openai
from qdrant_client import QdrantClient

### Embedding Function

In [4]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(input=text, model=model)
    return response.data[0].embedding

### Retrieval Function

In [30]:
qdrant_client_ = QdrantClient(url="http://localhost:6333")


def retrieve_data(query, qdrant_client_=qdrant_client_, k=5):
    query_embedding = get_embeddings(query)
    results = qdrant_client_.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    retrieved_context_ids = []
    retrieved_contexts = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_contexts.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "context_ids": retrieved_context_ids,
        "context": retrieved_contexts,
        "similarity_scores": similarity_scores,
    }

In [12]:
retrieve_data("What kind of tablets can i get?", k=10)

{'context_ids': ['B09QGNB537',
  'B0BF18F6R7',
  'B0BG5L2YLC',
  'B0BLV7YH2W',
  'B0C142QS8X',
  'B0CH6P8DYF',
  'B08BR9X387',
  'B09Y39DSWR',
  'B0CGM29ZL7',
  'B08BX2L8F2'],
 'context': ['ESR for iPad Air 5th Generation Case 2022 / iPad Air 4th Case 2020, Rugged Protection, Detachable Magnetic Cover, Adjustable Portrait/Landscape Stand with Raised Screen View, 8 Stand Angles, Black Compatibility: only compatible with iPad Air 5th Generation 2022 and iPad Air 4th Generation 2020; not compatible with any other iPad models Detachable Magnetic Cover: keep your screen covered, fold it up into a stand for viewing/writing/portrait modes, or remove it for better ergonomics while gaming Rugged Protection: rugged shock-resistant polymer provides robust protection against scratches and drops Stable and Adjustable Stand: magnetically locks to 8 different stand positions that stay stable in bed, on the couch, or anywhere else Stand Your Way: raised screen view, portrait mode, and magnetic fridge 

### format retrieved Context 

In [17]:
def process_context(context: dict):
    formatted_context = ""
    for id, chunk in zip(context["context_ids"], context["context"]):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context

In [14]:
retrieved_context = retrieve_data("What kind of tablets can i get?", k=10)

In [21]:
preprocessed_context = process_context(retrieved_context)

In [22]:
print(preprocessed_context)

- B09QGNB537: ESR for iPad Air 5th Generation Case 2022 / iPad Air 4th Case 2020, Rugged Protection, Detachable Magnetic Cover, Adjustable Portrait/Landscape Stand with Raised Screen View, 8 Stand Angles, Black Compatibility: only compatible with iPad Air 5th Generation 2022 and iPad Air 4th Generation 2020; not compatible with any other iPad models Detachable Magnetic Cover: keep your screen covered, fold it up into a stand for viewing/writing/portrait modes, or remove it for better ergonomics while gaming Rugged Protection: rugged shock-resistant polymer provides robust protection against scratches and drops Stable and Adjustable Stand: magnetically locks to 8 different stand positions that stay stable in bed, on the couch, or anywhere else Stand Your Way: raised screen view, portrait mode, and magnetic fridge mounting let you comfortably view and use your iPad in the office, kitchen, and beyond Complete Customer Support: detailed setup videos and FAQs, comprehensive 12-month warrant

### Create Prompt Function

In [24]:
def build_prompt(preprocessed_context, question):
    prompt = f"""
    You are a shopping assistant that can answer questions about the products in stock.

    You will be given a question and a list of context.

    Instructions:
    - You need to answer the question based on the provided context only.
    - Never use word context and refer to it as the available products.

    Context:
    {preprocessed_context}

    Question:
    {question}
    """

    return prompt

In [25]:
print(build_prompt(preprocessed_context, "what kind of tablets can i get?"))


    You are a shopping assistant that can answer questions about the products in stock.

    You will be given a question and a list of context.

    Instructions:
    - You need to answer the question based on the provided context only.
    - Never use word context and refer to it as the available products.

    Context:
    - B09QGNB537: ESR for iPad Air 5th Generation Case 2022 / iPad Air 4th Case 2020, Rugged Protection, Detachable Magnetic Cover, Adjustable Portrait/Landscape Stand with Raised Screen View, 8 Stand Angles, Black Compatibility: only compatible with iPad Air 5th Generation 2022 and iPad Air 4th Generation 2020; not compatible with any other iPad models Detachable Magnetic Cover: keep your screen covered, fold it up into a stand for viewing/writing/portrait modes, or remove it for better ergonomics while gaming Rugged Protection: rugged shock-resistant polymer provides robust protection against scratches and drops Stable and Adjustable Stand: magnetically locks to 8 

### Generate Answer Function

In [26]:
def generate_answer(prompt):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )
    return response.choices[0].message.content

In [28]:
prompt = build_prompt(preprocessed_context, "What kind of tablets can I get?")

print(generate_answer(prompt))

You can get cases compatible with the iPad Air 5th Generation (2022) and iPad Air 4th Generation (2020). Specifically, the ESR case offers rugged protection, a detachable magnetic cover, and an adjustable stand with 8 angles for these iPad Air models.


#### Combined RAG Pipeline

In [33]:
def rag_pipeline(question, topk=5):
    qdrant_client_ = QdrantClient(url="http://localhost:6333")
    retrieved_context = retrieve_data(question, qdrant_client_, topk)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)
    return answer

In [34]:
print(rag_pipeline("What kind of earphones can i get?"))

You can choose from several types of earphones available:

1. Open Ear Headphones: These rest gently on your ears without entering the ear canal, reducing stress and hearing loss. They offer immersive premium sound, a secure fit with silicone ear hooks, and are waterproof with up to 60 hours playtime with the charging case. Ideal for running, walking, and workouts.

2. Wireless In-Ear Earbuds: These Bluetooth 5.3 earbuds feature deep bass, a microphone, smart touch controls, LED power display, and 37 hours of playback time. They are ultra-light and IPX7 waterproof, suitable for sports.

3. Kids Over-Ear Headphones: Designed for children with volume-limiting technology (94dB) for hearing protection, adjustable padded headband, foldable design for portability, and compatible with all 3.5mm audio jack devices. They do not have a microphone.

4. Wireless Bluetooth Earbuds with Noise Cancelling Mic: These offer 30 hours playtime, clear call quality with dual microphones, deep bass sound, IP